## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [1]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

In [3]:
file_id = '1kmXAzIgGI0Yg6HM00pSL3VSXhAYidI6l'
download_file(file_id, 'cool_roof_yolo_labels.zip')
file_id = '1kisSOoULdWoLLo3Aoyel8jowar5gxKlN'
download_file(file_id, 'cool_roof_images.zip')
file_id = '1mh6wtj72ij9npCM8dy8kW2H1wGuu6oxl'
download_file(file_id, 'test.zip')

Downloading...
From: https://drive.google.com/uc?id=1kmXAzIgGI0Yg6HM00pSL3VSXhAYidI6l
To: /content/cool_roof_yolo_labels.zip
100%|██████████| 63.0k/63.0k [00:00<00:00, 41.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1kisSOoULdWoLLo3Aoyel8jowar5gxKlN
From (redirected): https://drive.google.com/uc?id=1kisSOoULdWoLLo3Aoyel8jowar5gxKlN&confirm=t&uuid=dc6bb3d9-2447-4a78-94e1-e99ba2b95638
To: /content/cool_roof_images.zip
100%|██████████| 216M/216M [00:02<00:00, 85.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mh6wtj72ij9npCM8dy8kW2H1wGuu6oxl
To: /content/test.zip
100%|██████████| 7.18M/7.18M [00:00<00:00, 15.1MB/s]


In [4]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [5]:
dataset_extract('cool_roof_yolo_labels.zip')
dataset_extract('cool_roof_images.zip')
dataset_extract('test.zip')

Extracting: 100%|██████████| 21/21 [00:00<00:00, 1002.91files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [6]:
#test,train,valid 폴더 생성
import os
path = '/content/cool_roof/'

for folder in ['train','test','valid']:
    os.makedirs(path+folder+'/images',exist_ok=True)
    os.makedirs(path+folder+'/labels',exist_ok=True)

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [7]:
from glob import glob
from sklearn.model_selection import train_test_split

image_path = glob('/content/cool_roof_images/*')
len(image_path)

200

In [8]:
train_img, valid_img, _, _ = train_test_split(image_path,image_path,test_size=0.2,random_state=2024)

len(train_img),len(valid_img)

(160, 40)

In [9]:
train_img[1].split('/')[-1].replace('jpg','txt')

'roof (68).txt'

In [10]:
train_img[1].replace('cool_roof_images','cool_roof_yolo_labels').replace('jpg','txt')

'/content/cool_roof_yolo_labels/roof (68).txt'

In [11]:
# train 파일 이동
for img_path in train_img:
    label_path = '/content/cool_roof_yolo_labels/obj_train_data/' + img_path.split('/')[-1].replace('jpg','txt')
    if os.path.exists(img_path):
        shutil.move(img_path,path+'train/images')
    if os.path.exists(label_path):
        shutil.move(label_path,path+'train/labels')
# valid 파일 이동
for img_path in valid_img:
    label_path = '/content/cool_roof_yolo_labels/obj_train_data/' + img_path.split('/')[-1].replace('jpg','txt')
    if os.path.exists(img_path):
        shutil.move(img_path,path+'valid/images')
    if os.path.exists(label_path):
        shutil.move(label_path,path+'valid/labels/')
# test 파일 이동
for img_path in glob('/content/test/test/*'):
    if os.path.exists(img_path):
        shutil.move(img_path,path+'test/images')

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [12]:
import yaml

yaml_data = {'train' : '/content/cool_roof/train/images',
               'val' : '/content/cool_roof/valid/images',
               'test' : '/content/cool_roof/test/images',
               'names' : ['cool roof', 'generic roof'],
               'nc' : 2 }


yaml_file_path = "/content/cool_roof/data.yaml"
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [13]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 8.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [14]:
from ultralytics import settings, YOLO

In [15]:
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [16]:
settings['datasets_dir'] = '/content/'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [19]:
model_transfer = YOLO(model='yolov8n.pt', task='detect')

In [20]:
model_transfer.train(data='/content/cool_roof/data.yaml',
                    epochs=100,
                    patience=10,
                    seed=2024,
                    pretrained=True,
                    hsv_h=0.008,
                    hsv_v=0.4
                    )

Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/cool_roof/data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /content/cool_roof/train/labels.cache... 160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 160/160 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/cool_roof/valid/labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         40        300      0.591      0.719      0.618      0.394
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 27, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



37 epochs completed in 0.024 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.42it/s]


                   all         40        300      0.646      0.767      0.766      0.482
             cool roof         40         39      0.635      0.713      0.735      0.474
          generic roof         40        261      0.656       0.82      0.797      0.489
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bd98ceaf730>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [21]:
image_path = '/content/cool_roof/test/images'
model_transfer.predict(source=image_path, save=True, iou=0.5, conf=0.5)


image 1/21 /content/cool_roof/test/images/KakaoTalk_20240411_143843284.png: 640x576 2 cool roofs, 14 generic roofs, 75.4ms
image 2/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112419.png: 576x640 2 generic roofs, 69.3ms
image 3/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112543.png: 608x640 2 generic roofs, 64.7ms
image 4/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112738.png: 640x640 1 generic roof, 8.8ms
image 5/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112832.png: 576x640 1 cool roof, 5 generic roofs, 10.9ms
image 6/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 112921.png: 640x640 1 cool roof, 2 generic roofs, 9.0ms
image 7/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 113028.png: 608x640 1 cool roof, 21 generic roofs, 9.1ms
image 8/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 113048.png: 576x640 8 generic roofs, 9.3ms
image 9/21 /content/cool_roof/test/images/╜║┼⌐╕░╝ª 2024-04-11 113117.png: 640x640 1 generic roof, 8

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'cool roof', 1: 'generic roof'}
 obb: None
 orig_img: array([[[ 38,  31,  14],
         [ 38,  31,  14],
         [ 39,  32,  15],
         ...,
         [ 50,  53,  31],
         [ 57,  60,  38],
         [ 61,  64,  42]],
 
        [[ 39,  32,  15],
         [ 40,  33,  16],
         [ 41,  34,  17],
         ...,
         [ 54,  57,  35],
         [ 60,  63,  41],
         [ 64,  67,  45]],
 
        [[ 46,  39,  22],
         [ 46,  39,  22],
         [ 47,  40,  23],
         ...,
         [ 54,  57,  35],
         [ 60,  63,  41],
         [ 63,  66,  44]],
 
        ...,
 
        [[166, 169, 143],
         [164, 167, 141],
         [163, 167, 139],
         ...,
         [ 54,  47,  28],
         [ 48,  41,  22],
         [ 44,  37,  18]],
 
        [[165, 168, 142],
         [163, 166, 140],
         [163, 167, 138],
         .

In [23]:
model_transfer.export(format='torchscript')

Ultralytics YOLOv8.1.46 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.30GHz)

PyTorch: starting from 'runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 1.4s, saved as 'runs/detect/train2/weights/best.torchscript' (11.9 MB)

Export complete (2.8s)
Results saved to /content/runs/detect/train2/weights
Predict:         yolo predict task=detect model=runs/detect/train2/weights/best.torchscript imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train2/weights/best.torchscript imgsz=640 data=/content/cool_roof/data.yaml  
Visualize:       https://netron.app


'runs/detect/train2/weights/best.torchscript'